# FlexibleBiommassPkg example

In [4]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
from cobra.flux_analysis.parsimonious import pfba
import cobrakbase
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import FlexibleBiomassPkg, ElementUptakePkg, KBaseMediaPkg


In [6]:
kbase_api = cobrakbase.KBaseAPI(token = "5YAOMNFB7XOEKBKHRHJB4AIZSQ2EJGA5")

model = kbase_api.get_from_ws("C4_Calvin_Zmays_Model",25427)
#model = kbase_api.get_from_ws("E_iML1515.kb",40576)
model.solver = 'optlang-cplex'

#Applying uptake limit to total carbon uptake by model
eup = ElementUptakePkg(model)
eup.build_package({"C":60})
#Applying constraints based on KBase media
#media = kbase_api.get_from_ws("Btheta_Ecoli_minimal_media",40576)
kmp = KBaseMediaPkg(model)
#kmp.build_package(media)
kmp.build_package(None)
#Applying flexible biomass constraints
fbp = FlexibleBiomassPkg(model)
fbp.build_package({"bio_rxn_id":"bio1","use_rna_class":[-0.75,0.75],
            "use_dna_class":[-0.75,0.75],
            "use_protein_class":[-0.1052,-0.1052],
            "use_energy_class":[-0.1,0.1]})
#Setting the objective to biomass
biomass_objective = model.problem.Objective(
    1 * model.reactions.bio1.flux_expression,
    direction='max')
model.objective = biomass_objective
#Printing LP file
with open('FlexibleBiomass.lp', 'w') as out:
    out.write(str(model.solver))
#Solving
sol=model.optimize()
model.summary()

rna:cpd00052_c0:480.13250600000003:-12.483445156:-0.026
rna:cpd00038_c0:520.156606:-26.007516912:-0.026
rna:cpd00062_c0:481.117266:-38.997683093999996:-0.027
rna:cpd00012_x0:174.951264:-25.176533237999994:0.079
rna:cpd00001_d1:18.01528:-23.753326117999993:0.079
dna:cpd00115_c0:488.157806:-14.644734179999999:-0.03
dna:cpd00356_c0:464.133106:-26.712194935999996:-0.026
dna:cpd00241_c0:504.157206:-39.820282291999995:-0.026
dna:cpd00357_c0:479.144446:-54.194615672:-0.03
dna:cpd00012_x0:174.951264:-34.600074104:0.11199999999999999
dna:cpd00001_d1:18.01528:-32.582362744:0.11199999999999999
protein:cpd00023_c0:146.12132:-18.9957716:-0.13
protein:cpd00033_c0:75.0666:-30.2557616:-0.15
protein:cpd00035_c0:89.09318:-44.59976358:-0.161
protein:cpd00039_c0:147.1955:-58.730531580000005:-0.096
protein:cpd00041_c0:132.09474:-71.6758161:-0.098
protein:cpd00051_c0:175.2089:-90.94879510000001:-0.11
protein:cpd00053_c0:146.1445:-101.32505460000002:-0.071
protein:cpd00054_c0:105.09258:-111.83431260000002:-0

In [3]:
for reaction in fbp.new_reactions:
    print(reaction+":"+str(fbp.new_reactions[reaction].forward_variable.primal-fbp.new_reactions[reaction].reverse_variable.primal))

FLEX_cpd00201_c0:-3.450255666989782e-05
FLEX_cpd24604_c0:-4.022719611736966e-06
FLEX_cpd03445_c0:-3.450255666989782e-05
FLEX_cpd24682_c0:-4.022719611736965e-05
FLEX_cpd00035_c0:0.0794779544089827
FLEX_cpd00017_c0:-3.450255666989782e-05
FLEX_cpd00051_c0:-0.04576493382288079
FLEX_cpd00132_c0:0.03729602600027901
FLEX_cpd00041_c0:0.03729602600027901
FLEX_cpd00104_c0:-3.0943997013361277e-07
FLEX_cpd00063_c0:0.0008053175222727272
FLEX_cpd00099_c0:0.0008053175222727272
FLEX_cpd00010_c0:-8.911871139848047e-05
FLEX_cpd00149_c0:3.86799962667016e-06
FLEX_cpd00052_c0:0.020656355766299185
FLEX_cpd00058_c0:0.00010969646941236572
FLEX_cpd00084_c0:-0.014169256232418128
FLEX_cpd00115_c0:-0.0040484031292580555
FLEX_cpd00356_c0:0.004180069836549908
FLEX_cpd00241_c0:-0.004180069836549908
FLEX_cpd00015_c0:-3.450255666989782e-05
FLEX_cpd10515_c0:0.0010389446997236048
FLEX_cpd10516_c0:0.0012080536434016242
FLEX_cpd00053_c0:0.040716111270180766
FLEX_cpd00023_c0:0.040716111270180766
FLEX_cpd00033_c0:-0.0947873